<a href="https://colab.research.google.com/github/ErnestoAvedillo/AAM-math/blob/main/AAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

**### Primera parte**

Cálculo del coste de las demandas internas.

n = nḿero de referencias.

REF = diccionario con las referencias de la empresa (TODAS , las de compra , internas  y de producto acabado.)

DC = Demanda de cliente

BOM = es una tabla de n x n con el BOM de los producto (Todos) Build of Materials

In [3]:
#numero de referencias
n = 8
#BILL OF MATERIALS
BOM = np.zeros((n,n))
#Diccionario de referencias
#REF = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6,"8":7}
REF = {}
for i in range(0,n):
  num = str(i + 1)
  REF[num] = i
#Demandas de cliente
Dias = 5
Profundidad = 5
DC = np.zeros((n,Profundidad,Dias))

Simulacion de un BOM con dos productos diferentes

In [4]:
#BOM for product 1
BOM[REF["5"],REF["3"]] = 0.5
BOM[REF["5"],REF["7"]] = 2
BOM[REF["7"],REF["1"]] = 1

#BOM for product 2
BOM[REF["6"],REF["3"]] = 0.75
BOM[REF["6"],REF["4"]] = 1
BOM[REF["6"],REF["8"]] = 1
BOM[REF["8"],REF["2"]] = 1

print(BOM)
print(REF)

[[0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.5  0.   0.   0.   2.   0.  ]
 [0.   0.   0.75 1.   0.   0.   0.   1.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.  ]]
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}


In [5]:
#DEMANDAS DE CLIENTE
DC[REF["5"],0,0]=700
DC[REF["6"],0,0]=1000
DC[REF["5"],0,1]=0
DC[REF["6"],0,1]=2000
DC[REF["5"],0,2]=2000
DC[REF["6"],0,2]=0
DC[REF["5"],0,3]=7000
DC[REF["6"],0,3]=1000
DC[REF["5"],0,4]=7
DC[REF["6"],0,4]=1000
print (DC)

[[[   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]]

 [[   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]]

 [[   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]]

 [[   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]]

 [[ 700.    0. 2000. 7000.    7.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]]

 [[1000. 2000.    0. 1000. 1000.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.]
  [ 

Listado de Maquinaria

m = número de máquinas

DICMAQ = Diccionario de maquinaria (m x 1)

BOMMAQ = Listado de maquinaria por referencia (m x n)

In [6]:
numMaquinas = 3
BOMMAQ = np.zeros((numMaquinas,n))
DICMAQ = {"Compras":0,"mecanizado":1,"inyeccion":2}
#BOMMAQ[DICMAQ["Compras"],REF["1"]] = 1
#BOMMAQ[DICMAQ["Compras"],REF["2"]] = 1
#BOMMAQ[DICMAQ["Compras"],REF["3"]] = 1
#BOMMAQ[DICMAQ["Compras"],REF["4"]] = 1
BOMMAQ[DICMAQ["inyeccion"],REF["5"]] = 1
BOMMAQ[DICMAQ["inyeccion"],REF["6"]] = 1
BOMMAQ[DICMAQ["mecanizado"],REF["7"]] = 1
BOMMAQ[DICMAQ["mecanizado"],REF["8"]] = 1
print(BOMMAQ)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0.]]


In [7]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
for dayIndex in range(Dias):
  for profIndex in range(Profundidad-1):
    DC[:,profIndex+1,dayIndex] = np.dot(BOM.T,DC[:,profIndex,dayIndex])
for dayIndex in range(Dias):
  print(DC[:,:,dayIndex],'\n')

[[   0.    0. 1400.    0.    0.]
 [   0.    0. 1000.    0.    0.]
 [   0. 1100.    0.    0.    0.]
 [   0. 1000.    0.    0.    0.]
 [ 700.    0.    0.    0.    0.]
 [1000.    0.    0.    0.    0.]
 [   0. 1400.    0.    0.    0.]
 [   0. 1000.    0.    0.    0.]] 

[[   0.    0.    0.    0.    0.]
 [   0.    0. 2000.    0.    0.]
 [   0. 1500.    0.    0.    0.]
 [   0. 2000.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [2000.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0. 2000.    0.    0.    0.]] 

[[   0.    0. 4000.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0. 1000.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [2000.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0. 4000.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]] 

[[    0.     0. 14000.     0.     0.]
 [    0.     0.  1000.     0.     0.]
 [    0.  4250.     0.     0.     0.]
 [    0.  1000.     0.     0.     0.]
 [ 7000.     0.     0.     0.     0.]
 [ 1000. 

In [8]:
#DINT se corresponde a las demandas internas
for i in list(REF):
  print("Referencia " ,i, " Demanda CLIENTE --> ", DC[REF[i],0,:])
for j in range(1, DC.shape[1] - 1):
  for i in list(REF):
    print("Referencia " ,i, " Demanda interna nivel ", j, " --> ", DC[REF[i],j,:])
for i in list(REF):
  print("Referencia " ,i, " Demanda interna (compras) nivel 0 --> ", DC[REF[i],DC.shape[1] - 1,:])


Referencia  1  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  2  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  3  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  4  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  5  Demanda CLIENTE -->  [ 700.    0. 2000. 7000.    7.]
Referencia  6  Demanda CLIENTE -->  [1000. 2000.    0. 1000. 1000.]
Referencia  7  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  8  Demanda CLIENTE -->  [0. 0. 0. 0. 0.]
Referencia  1  Demanda interna nivel  1  -->  [0. 0. 0. 0. 0.]
Referencia  2  Demanda interna nivel  1  -->  [0. 0. 0. 0. 0.]
Referencia  3  Demanda interna nivel  1  -->  [1100.  1500.  1000.  4250.   753.5]
Referencia  4  Demanda interna nivel  1  -->  [1000. 2000.    0. 1000. 1000.]
Referencia  5  Demanda interna nivel  1  -->  [0. 0. 0. 0. 0.]
Referencia  6  Demanda interna nivel  1  -->  [0. 0. 0. 0. 0.]
Referencia  7  Demanda interna nivel  1  -->  [ 1400.     0.  4000. 14000.    14.]
Referencia  8  Demanda interna nivel  1  -->  [100

Ahora vamos a buscar el minimo común múltiplo entre las demandas y las piezas por caja
Posteriormente calcularemos el tiempo de trabajo necesario para cada un o de los niveles de producción.

T =  Tiempos de ciclo (n x 1) (Dato a recibir de cliente)

BOX = Piezas por caja (n x 1)

In [9]:
#Tiempo de ciclo estimado en los procesos
T= np.zeros((n, 1))
T[REF["5"]] = 120
T[REF["6"]] = 135
T[REF["7"]] = 90
T[REF["8"]] = 80
print(T)

[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [120.]
 [135.]
 [ 90.]
 [ 80.]]


In [10]:
#Estimo las piezs por caja de cada componente
import random
BOX = np.ones((n,1))
for i in range(n):
  BOX[i - 1] = random.randrange(5,10)
print (BOX)

[[7.]
 [8.]
 [9.]
 [8.]
 [5.]
 [8.]
 [6.]
 [9.]]


In [11]:
#Busco en número de cajas necesario para la produccion
DCBOX = np.copy(DC);
for dayIndex in range(Dias):
  mask = np.mod(DC[:,:,dayIndex], BOX) > 0
  ones_array = np.zeros_like(np.mod(DC[:,:,dayIndex], BOX))
  ones_array [mask] = 1
  DCBOX[:,:,dayIndex] = np.floor_divide(DC[:,:,dayIndex], BOX) +  ones_array
  print (DCBOX[:,:,dayIndex],'\n')


[[  0.   0. 200.   0.   0.]
 [  0.   0. 125.   0.   0.]
 [  0. 123.   0.   0.   0.]
 [  0. 125.   0.   0.   0.]
 [140.   0.   0.   0.   0.]
 [125.   0.   0.   0.   0.]
 [  0. 234.   0.   0.   0.]
 [  0. 112.   0.   0.   0.]] 

[[  0.   0.   0.   0.   0.]
 [  0.   0. 250.   0.   0.]
 [  0. 167.   0.   0.   0.]
 [  0. 250.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [250.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0. 223.   0.   0.   0.]] 

[[  0.   0. 572.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0. 112.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [400.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0. 667.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]] 

[[   0.    0. 2000.    0.    0.]
 [   0.    0.  125.    0.    0.]
 [   0.  473.    0.    0.    0.]
 [   0.  125.    0.    0.    0.]
 [1400.    0.    0.    0.    0.]
 [ 125.    0.    0.    0.    0.]
 [   0. 2334.    0.    0.    0.]
 [   0.  112.    0.    0.    0.]] 

[[  0.   0.   2.   0.   0.]
 [  0.   0. 125.   0.   

In [12]:
DCHORAS = np.copy(DC);
for dayIndex in range(Dias):
  DCHORAS[:,:,dayIndex] = np.multiply(np.multiply(DCBOX[:,:,dayIndex],T),BOX)/3600
  print(DCHORAS[:,:,dayIndex],'\n')

[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [23.33333333  0.          0.          0.          0.        ]
 [37.5         0.          0.          0.          0.        ]
 [ 0.         35.1         0.          0.          0.        ]
 [ 0.         22.4         0.          0.          0.        ]] 

[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [75.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.  44.6  0.   0.   0. ]] 

[[  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.    


MAQ = is a vector of ONES where there is an operation


In [13]:
MAQ = np.zeros((numMaquinas,Dias));
for dayIndex in range(Dias):
  MAQ[:,dayIndex] = np.dot(DCHORAS[:,:,dayIndex].sum(axis = 1), BOMMAQ.T)
  print(MAQ[:,dayIndex],'\n')

[ 0.         57.5        60.83333333] 

[ 0.  44.6 75. ] 

[  0.         100.05        66.66666667] 

[  0.         372.5        270.83333333] 

[ 0.         22.85       37.83333333] 



# Inicio de la simulacion
## Constraints de la simulacion

LoteMin = Lote mínimo de producción (n x 1)

LotMax = Lote máximo de producción (n x 1)

TCambio = Tiempos de cambio de referencia (n x n)

Planif =  Planificacion de la producción (Maq x 24 horas x Ref)

PrioMat = Matriz de priodidades (n x 1)

In [28]:
print (MAQ)
print (DCHORAS[:,:,0])

[[  0.           0.           0.           0.           0.        ]
 [ 57.5         44.6        100.05       372.5         22.85      ]
 [ 60.83333333  75.          66.66666667 270.83333333  37.83333333]]
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [23.33333333  0.          0.          0.          0.        ]
 [37.5         0.          0.          0.          0.        ]
 [ 0.         35.1         0.          0.          0.        ]
 [ 0.         22.4         0.          0.          0.        ]]


# Cálculo de costes del proceso

En esta sección calcularemos los costes del proceso.

En esencia el cálculo es el siguiente:

Coste proceso [CP] = Coste mano de obra[CMO] + Coste Máquina [CMQ]

Desarrollando un poco mas este cálculo tenemos que:

 CMO = Coste operario [MO] / Tiempo de ciclo [T] / 3600

CMQ = Coste Máquina fijo [CMQF] Coste Máquina Variable  [CMQV]

CMQF = Coste fijo Máquina [CF] * T / 3600

CMQV = Coste potencia [CPOT] * Potencia Máquina [POT] * T / 3600

Sustituyento tenemos:

CP = (MO + CF + CPOT * POT) * T / 3600

Ahora nos centraremso en el cálculo de la mano de obra

MAQREF = Vector Maquinas (n x 1)

MOM = Coste de la mano de obra Turno de mañana

MOT = Coste de la mano de obra Turno de tarde

MON = Coste de la mano de obra Turno de noche

MHORARIA = MATRIZ Horaria (n x 24)

NOP =  Número de operarios (n x 1) (Dato a recibir de cliente)

CMOTHORARIO = Coste de mano de obra total (nx24) resultado de MOTURNOS * MOP *T / 3600

In [15]:
MAQREF = np.zeros((n,1))
mask = BOMMAQ.sum(axis = 0) > 0
MAQREF[mask] = 1
print(MAQREF)
MHORARIA = np.ones((n,24))
MOTURNOS = np.multiply(MHORARIA, MAQREF)
print(MOTURNOS)

[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [16]:
MOM = 12
MOT = 14
MON = 18
MOTURNOS [:,0:6] *=  MON
MOTURNOS [:,6:14] *=  MOM
MOTURNOS [:,14:22] *=  MOT
MOTURNOS [:,22:24] *=  MON
print(MOTURNOS)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]]


In [17]:
#Numero de operarios estimado en los procesos
NOP= np.copy(MAQREF)
NOP[REF["5"]] = 2
NOP[REF["6"]] = 3
NOP[REF["7"]] = 1
NOP[REF["8"]] = 1
print(NOP)

[[0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [3.]
 [1.]
 [1.]]


In [18]:
CMOTHORARIO  =np.multiply(MOTURNOS, np.multiply(T,NOP)) / 3600
print(CMOTHORARIO)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.


Ahora nos centraremso en el cálculo de la mano de obra

MAQ = Vector Maquinas (n x 1)

CMF = Coste máquina fijo (n x 24)

CPOT = Tabla de coste de potencias (n x 24)

POT = Potencia de Maquinaria (n x 1) KWH

CELECM = Coste potencia turno mañana

CELECT = coste potencia turno tarde

CELECN = Coste potencia turno Noche

CMVTURNOS = Coste maquina variable (n x 24))

T =  Tiempos de ciclo (n x 1) (Definidos anteriormente)

CMQVTHORARIO = Coste de maquina total (nx24) resultado de CMVTURNOS * POT *T / 3600

CMTOT = (CMF + CMQVTHORARIO) * T / 3600

In [19]:
from genericpath import commonprefix
#Coste maquina definido por cliente CMF
cmfijo = 5  #5 es un valor estimado
CMF = np.ones((n,24))
CMF = np.multiply(CMF, MAQREF)
CMF = CMF * cmfijo
print (CMF)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]]


In [20]:
#Numero de operarios estimado en los procesos
CPOT= np.copy(MAQREF)
CPOT[REF["5"]] = 50
CPOT[REF["6"]] = 60
CPOT[REF["7"]] = 120
CPOT[REF["8"]] = 12
print(CPOT)

[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [ 50.]
 [ 60.]
 [120.]
 [ 12.]]


In [21]:
CMTURNOS = np.ones((n,24))
CMTURNOS = np.multiply(CMTURNOS, MAQREF)
print(CMTURNOS)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [22]:

CELECM = 0.12
CELECT = 0.14
CELECN = 0.16
CMTURNOS [:,0:6] *=  CELECN
CMTURNOS [:,6:14] *=  CELECM
CMTURNOS [:,14:22] *=  CELECT
CMTURNOS [:,22:24] *=  CELECN
print(CMTURNOS)

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]]


In [23]:
CMQVTHORARIO  = np.multiply(CMTURNOS , CPOT)
print(CMQVTHORARIO)

[[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 8.    8.    8.    8.    8.    8.    6.    6.    6.    6.    6.    6.
   6.    6.    7.    7.    7.    7.    7.    7.    7.    7.    8.    8.  ]
 [ 9.6   9.6   9.6   9.6   9.6   9.6   7.2   7.2   7.2   7.2   7.2   7.2
   7.2   7.2   8.4   8.4   8.4   8.4   8.4   8.4   8.4   8.4   9.6   9.6 ]
 [19.2  19.2  19.2  19.2  19.2  19.2  14.4  14.4  14.4  14.4  14.4  14.4
  14.4  14.4  16.8  16.8  16.8  16.8  16.8  

In [24]:
CMTOT = np.multiply((CMF + CMQVTHORARIO), T) / 3600
print (CMTOT)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.

El coste total es el resultado de sumar el coste máquina y el de operario
CTOTHORARIO

In [25]:
CTOTHORARIO = CMOTHORARIO + CMTOT
print (CTOTHORARIO)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.